In [1]:
training_data=[['Green',3,'Apple'],
               ['Yellow',3,'Apple'],
               ['Red',1,'Grape'],
               ['Red',1,'Grape'],
               ['Yellow',3,'Lemon']
              ]
header=['Color','Diameter','Fruit']
def unique_vals(Data,col):
    return set([row[col] for row in Data])
print(unique_vals(training_data,1))
def count_labels(Data):
    counts={}
    for row in Data:
        label=row[-1]
        if label not in counts:
            counts[label]=0
        counts[label]+=1
    return counts
count_labels(training_data)
def is_numeric(value):
    return isinstance(value,int) or isinstance(value,float)
is_numeric(20)
count_labels(training_data)

{1, 3}


{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [5]:
class Question:
    def __init__(self,column,value):
        self.column=column
        self.value=value
        
    def match(self,example):
        val=example[self.column]
        if is_numeric(val):
            return val ==self.value
        else:
            return val==self.value
        
    def __repr__(self):
        #This is just a helper method to print the question formed
        #the question in a readable formt.
        condition="=="
        if is_numeric(self.value):
            condition="=="
        return "Is %s %s %s ?" %(header[self.column],condition,str(self.value))
Question(0,'Red')


Is Color == Red ?

In [6]:
def partition(rows,question):
    true_rows,false_rows=[],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows , false_rows    
            

In [7]:
true_rows,false_rows=partition(training_data, Question(1,1))
print(true_rows)

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]


In [8]:
def gini(rows):
    counts=count_labels(rows)
    impurity=1
    for lbl in counts:
        prob_of_lbl=counts[lbl]/float(len(rows))
        impurity -=prob_of_lbl**2
    return impurity



In [9]:
def info_gain(left, right, current_uncertanity):
    p=float(len(left)) / (len(left)+len(right))
    q=float(len(right)) / (len(left)+len(right))
    return current_uncertanity - p * gini(left) - q * gini(right)


In [10]:
true_rows, false_rows = partition(training_data,Question(1,3))
info_gain(true_rows,false_rows,0.84)
gini(training_data)


0.6399999999999999

In [11]:
def find_best_split(rows):
    best_gain =0
    best_question = None
    current_uncertanity = gini(rows)
    n_features = len(rows[0])-1
    for col in range(n_features):
        values=set(row[col] for row in rows)
        for val in values:
            question=Question(col,val)
            true_rows,false_rows=partition(rows,question)
            if(len(true_rows))==0 or len(false_rows)==0:
                continue
            gain= info_gain(true_rows,false_rows,current_uncertanity)
            if gain>=best_gain:
                best_gain , best_question = gain , question
    return best_gain , best_question    
            

In [12]:
class Leaf:
    def __init__(self,rows):
        self.predictions=count_labels(rows)

In [13]:
class Decision_Node:
    def __init__(self,question,true_branch,false_branch):
        self.question=question
        self.true_branch=true_branch
        self.false_branch=false_branch
        

In [14]:
best_gain,best_question=find_best_split(training_data)
print(best_gain,best_question)

0.37333333333333324 Is Diameter == 3 ?


In [15]:
def build_tree(rows):
    gain,question = find_best_split(rows)
    if gain==0:
        return Leaf(rows)
    true_rows , false_rows=partition(rows,question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question,true_branch,false_branch)

In [16]:
mytree=build_tree(training_data)

In [17]:
def print_tree(node,spacing=""):
    if isinstance(node,Leaf):
        print(spacing+"Predict",node.predictions)
        return
    #print the question at this node
    print(spacing+str(node.question))
    #call this function recursively on the true branch
    print(spacing+'--> True:')
    print_tree(node.true_branch,spacing+" ")
    
    #call this function recursively on the fasle branch
    print(spacing+'--> False:')
    print_tree(node.false_branch,spacing+" ")

In [18]:
print_tree(mytree)

Is Diameter == 3 ?
--> True:
 Is Color == Green ?
 --> True:
  Predict {'Apple': 1}
 --> False:
  Predict {'Apple': 1, 'Lemon': 1}
--> False:
 Predict {'Grape': 2}


In [19]:
header=['Day','Outlook','Temperature','Humidity','Wind','PlayTennis']
data=[['Sunny','Hot','High','Weak','No'],
['Sunny','Hot','High','Strong','No'],
['Overcast','Hot','High','Weak','Yes'],
['Rain','Mild','High','Weak','Yes'],
['Rain','Cool','Normal','Weak','Yes'],
['Rain','Cool','Normal','Strong','No'],
['Overcast','Cool','Normal','Strong','Yes'],
['Sunny','Mild','High','Weak','No'],
['Sunny','Cool','Normal','Weak','Yes'],
['Rain','Mild','Normal','Weak','Yes'],
['Sunny','Mild','Normal','Strong','Yes'],
['Overcast','Mild','High','Strong','Yes'],
['Overcast','Hot','Normal','Weak','Yes'],
['Rain','Mild','High','Strong','No'],]

tennis_tree=build_tree(data)
print_tree(tennis_tree)

Is Day == Overcast ?
--> True:
 Predict {'Yes': 4}
--> False:
 Is Temperature == High ?
 --> True:
  Is Day == Rain ?
  --> True:
   Is Humidity == Weak ?
   --> True:
    Predict {'Yes': 1}
   --> False:
    Predict {'No': 1}
  --> False:
   Predict {'No': 3}
 --> False:
  Is Humidity == Weak ?
  --> True:
   Predict {'Yes': 3}
  --> False:
   Is Outlook == Cool ?
   --> True:
    Predict {'No': 1}
   --> False:
    Predict {'Yes': 1}
